delete 'gb' index

In [1]:
curl -XDELETE 'localhost:9200/gb?pretty'

{
  "acknowledged" : true
}


create new 'gb' index, and specify mappings w/ analyzer

In [2]:
curl -XPUT 'localhost:9200/gb?pretty' -H 'Content-Type: application/json' -d'
{
  "mappings": {
    "tweet" : {
      "properties" : {
        "tweet" : {
          "type" :    "string",
          "analyzer": "english"
        },
        "date" : {
          "type" :   "date"
        },
        "name" : {
          "type" :   "string"
        },
        "user_id" : {
          "type" :   "long"
        }
      }
    }
  }
}
'

{
  "acknowledged" : true,
  "shards_acknowledged" : true
}


add new 'tag' field which we don't want to analyze

In [3]:
curl -XPUT 'localhost:9200/gb/_mapping/tweet?pretty' -H 'Content-Type: application/json' -d'
{
  "properties" : {
    "tag" : {
      "type" :    "string",
      "index":    "not_analyzed"
    }
  }
}
'

{
  "acknowledged" : true
}


test analyze 'tweet'

In [5]:
curl -XGET 'localhost:9200/gb/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "field": "tweet",
  "text": "Black-cats" 
}
'

{
  "tokens" : [
    {
      "token" : "black",
      "start_offset" : 0,
      "end_offset" : 5,
      "type" : "<ALPHANUM>",
      "position" : 0
    },
    {
      "token" : "cat",
      "start_offset" : 6,
      "end_offset" : 10,
      "type" : "<ALPHANUM>",
      "position" : 1
    }
  ]
}


test analyze 'tag' (should fail!)

In [6]:
curl -XGET 'localhost:9200/gb/_analyze?pretty' -H 'Content-Type: application/json' -d'
{
  "field": "tag",
  "text": "Black-cats" 
}
'

{
  "error" : {
    "root_cause" : [
      {
        "type" : "remote_transport_exception",
        "reason" : "[node-2][127.0.0.1:9301][indices:admin/analyze[s]]"
      }
    ],
    "type" : "illegal_argument_exception",
    "reason" : "Can't process field [tag], Analysis requests are only supported on tokenized fields"
  },
  "status" : 400
}
